# Tiny few shot classifer

Tiny Classifier nach dem Vorbild von: https://huggingface.co/blog/setfit

Lade 1. Bibliotheken: 

- `from datasets import load_dataset`: Zum Laden von Datensätzen aus der HuggingFace `datasets` Bibliothek, z.B. öffentliche oder lokale Datensätze.
  
- `from sentence_transformers.losses import CosineSimilarityLoss`: Wird verwendet, um einen *Cosine Similarity Loss* für das Training von sentence-transformers-Modellen zu definieren (z.B. bei Ähnlichkeitsvergleichen zwischen Textpaaren).
  
- `from setfit import SetFitModel, SetFitTrainer`:  
  - **SetFitModel**: Ein Modell aus dem SetFit-Framework, das für Zero-Shot- oder Few-Shot-Learning verwendet wird.
  - **SetFitTrainer**: Trainer-Klasse, die verwendet wird, um das SetFit-Modell auf spezifische Aufgaben zu trainieren.
  
- `import pandas as pd`: Für die Data-Wrangling
  
- `from sklearn.preprocessing import LabelEncoder`: Zum Kodieren von Zielvariablen (Labels) in numerische Form. Besonders nützlich, wenn die Labels in string-Format vorliegen und in Zahlen umgewandelt werden müssen, um sie einem Modell zu übergeben.
  
- `import os`: Ermöglicht die Interaktion mit dem Betriebssystem, wie z.B. den Zugriff auf Dateipfade, Erstellen von Ordnern oder Lesen von Umgebungsvariablen.
  
- `from datasets import Dataset, DatasetDict`:  
  - **Dataset**: Klasse, um einen benutzerdefinierten Datensatz für die Verwendung mit der `datasets` Bibliothek zu erstellen.
  - **DatasetDict**: Zum Organisieren und Verwalten mehrerer Datensätze (z.B. `train` und `test`).
  
- `from sklearn.model_selection import train_test_split`: Zum Aufteilen eines Datensatzes in Trainings- und Testdaten. Diese Methode wird oft verwendet, um Modelle zu validieren und zu evaluieren.
  
- `from transformers import TrainingArguments`: Legt die Trainingsparameter für Modelle aus der HuggingFace Transformers Bibliothek fest (z.B. Anzahl der Epochen, Batch-Größe, Optimierungsparameter etc.).


In [5]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

from transformers import TrainingArguments

# checke home-verzeichnis
print(os.getcwd())
# # setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification


Lade Datensatz:

In [6]:
# Laden Ihrer Excel-Datei
df = pd.read_excel('data/future_skill_examples.xlsx')
df

,label,label_text,group_label,group_label_text,label_text_desc,text_template_insert,text
0,0,Digital Literacy,0,Digitale Schlüsselkompetenzen,Beherrschen von grundlegenden digitalen Fähigk...,Digital Literacy,Die Veranstaltung behandelt das Thema Digital ...
1,0,Digital Literacy,0,Digitale Schlüsselkompetenzen,Beherrschen von grundlegenden digitalen Fähigk...,grundlegende digitale Fähigkeiten,Die Veranstaltung behandelt das Thema grundleg...
2,0,Digital Literacy,0,Digitale Schlüsselkompetenzen,Beherrschen von grundlegenden digitalen Fähigk...,grundlegenden Sicherheitsregeln im Netz,Die Veranstaltung behandelt das Thema grundleg...
3,0,Digital Literacy,0,Digitale Schlüsselkompetenzen,Beherrschen von grundlegenden digitalen Fähigk...,Nutzung gängiger Software,Die Veranstaltung behandelt das Thema Nutzung ...
4,0,Digital Literacy,0,Digitale Schlüsselkompetenzen,Beherrschen von grundlegenden digitalen Fähigk...,Benutzung digitaler Technologien,Die Veranstaltung behandelt das Thema Benutzun...
...,...,...,...,...,...,...,...
185,20,Missionsorientierung,3,Transformative Kompetenzen,Entwicklung einer Mission; Schaffung eines Mis...,"Fähigkeit, Menschen zu inspirieren und zu bewegen",Die Veranstaltung behandelt das Thema Fähigkei...
186,20,Missionsorientierung,3,Transformative Kompetenzen,Entwicklung einer Mission; Schaffung eines Mis...,Sinn stiften,Die Veranstaltung behandelt das Thema Sinn sti...
187,20,Missionsorientierung,3,Transformative Kompetenzen,Entwicklung einer Mission; Schaffung eines Mis...,Orientierung geben,Die Veranstaltung behandelt das Thema Orientie...
188,20,Missionsorientierung,3,Transformative Kompetenzen,Entwicklung einer Mission; Schaffung eines Mis...,Überzeugungskraft,Die Veranstaltung behandelt das Thema Überzeug...


erstelle test- und train-data:

In [7]:

# Aufteilen des DataFrames
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Konvertieren in Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Erstellen eines DatasetDicts
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


Modell und Trainer initialisieren:

In [8]:
from setfit import SetFitModel, SetFitTrainer

model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Trainer konfigurieren:

In [9]:
from sentence_transformers.losses import CosineSimilarityLoss

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     evaluation_strategy="epoch",  # Hier das korrekte Attribut verwenden
#     num_train_epochs=1,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     warmup_steps=10,
#     weight_decay=0.01,
#     logging_dir="test_trainer/logs",
#     logging_steps=10,
# )

# Initialisieren des Trainers mit den Trainingsargumenten
trainer = SetFitTrainer(
    model=model, 
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20,  # Anzahl der Textpaarungen pro Beispiel
    num_epochs=1        # Anzahl der Epochen
)



C:\Users\Hueck\AppData\Local\Temp\ipykernel_9096\943071219.py:16: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
c:\Users\Hueck\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\utils\_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]


Map:   0%|          | 0/152 [00:00<?, ? examples/s]

In [10]:
trainer.train()


***** Running training *****
  Num unique pairs = 6080
  Batch size = 16
  Num epochs = 1


  0%|          | 0/380 [00:00<?, ?it/s]

{'embedding_loss': 0.1949, 'grad_norm': 1.2970526218414307, 'learning_rate': 5.263157894736843e-07, 'epoch': 0.0}
{'embedding_loss': 0.2019, 'grad_norm': 1.9725565910339355, 'learning_rate': 1.929824561403509e-05, 'epoch': 0.13}
{'embedding_loss': 0.1483, 'grad_norm': 1.1298645734786987, 'learning_rate': 1.6374269005847955e-05, 'epoch': 0.26}
{'embedding_loss': 0.118, 'grad_norm': 0.663077712059021, 'learning_rate': 1.345029239766082e-05, 'epoch': 0.39}
{'embedding_loss': 0.0913, 'grad_norm': 1.422755479812622, 'learning_rate': 1.0526315789473684e-05, 'epoch': 0.53}
{'embedding_loss': 0.0753, 'grad_norm': 1.8479235172271729, 'learning_rate': 7.60233918128655e-06, 'epoch': 0.66}
{'embedding_loss': 0.0671, 'grad_norm': 0.8991213440895081, 'learning_rate': 4.678362573099415e-06, 'epoch': 0.79}
{'embedding_loss': 0.0677, 'grad_norm': 0.7197310924530029, 'learning_rate': 1.7543859649122807e-06, 'epoch': 0.92}


Computing widget examples:   0%|          | 0/5 [00:00<?, ?example/s]

{'train_runtime': 61.4454, 'train_samples_per_second': 98.95, 'train_steps_per_second': 6.184, 'train_loss': 0.10616480280694209, 'epoch': 1.0}


In [11]:
import datasets
metrics = trainer.evaluate()

metrics

***** Running evaluation *****


{'accuracy': 0.39473684210526316}

In [12]:
metrics

{'accuracy': 0.39473684210526316}